In [1]:
#| include: false

import duckdb
import pandas as pd
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

%sql CREATE OR REPLACE TABLE user_actions AS SELECT * FROM read_csv('00_data/sql/user_actions.csv', header=True, columns={'user_id': 'INT', 'order_id': 'INT', 'action': 'VARCHAR', 'time': 'TIMESTAMP'}, timestampformat='%d/%m/%y %H:%M');
%sql CREATE OR REPLACE TABLE courier_actions AS SELECT * FROM read_csv('00_data/sql/courier_actions.csv', header=True, columns={'courier_id': 'INT', 'order_id': 'INT', 'action': 'VARCHAR', 'time': 'TIMESTAMP'}, timestampformat='%d/%m/%y %H:%M');
%sql CREATE OR REPLACE TABLE orders AS SELECT * FROM read_csv('00_data/sql/orders.csv', header=True, columns={'order_id': 'INT', 'creation_time': 'TIMESTAMP', 'product_ids': 'INT[]'}, timestampformat='%d/%m/%y %H:%M');
%sql CREATE OR REPLACE TABLE users AS SELECT * FROM read_csv('00_data/sql/users.csv', header=True, columns={'user_id': 'INT', 'birth_date': 'DATE', 'sex': 'VARCHAR'}, dateformat='%d/%m/%y');
%sql CREATE OR REPLACE TABLE couriers AS SELECT * FROM read_csv('00_data/sql/couriers.csv', header=True, columns={'courier_id': 'INT', 'birth_date': 'DATE', 'sex': 'VARCHAR'}, dateformat='%d/%m/%y');
%sql CREATE OR REPLACE TABLE products AS SELECT * FROM read_csv('00_data/sql/products.csv', header=True, columns={'product_id': 'INT', 'name': 'VARCHAR', 'price': 'DOUBLE'});

,Count
0,87


# Підзапити

## Порядок виконання запитів

Перш ніж переходити до підзапитів, давайте трохи поговоримо про порядок виконання запитів у SQL.

До цього моменту ми вже встигли познайомитися з основними операторами, які складають "скелет" стандартного SQL-запиту: `SELECT`, `FROM`, `WHERE`, `GROUP BY`, `HAVING`, `ORDER BY` та `LIMIT`.

Ми вже знаємо, що порядок їх написання у запиті наступний:

```sql
SELECT      -- перелік полів результуючої таблиці
FROM        -- вказівник джерела даних
WHERE       -- фільтрація даних
GROUP BY    -- групування даних
HAVING      -- фільтрація даних після групування
ORDER BY    -- сортування результуючої таблиці
LIMIT       -- обмеження кількості записів у виводі
```

Проте важливо розуміти, що порядок виконання операторів у базах даних дещо відрізняється від порядку їх написання у запиті. У спрощеному вигляді порядок виконання запиту такий:

```sql
FROM       -- вказівник джерела даних
WHERE      -- фільтрація даних
GROUP BY   -- групування даних
HAVING     -- фільтрація даних після групування
SELECT     -- перелік полів результуючої таблиці
ORDER BY   -- сортування результуючої таблиці
LIMIT      -- бмеження кількості записів у виводі
```

Таким чином:

1. Спочатку з допомогою `FROM` визначається таблиця.
2. Потім відповідно до зазначеної у `WHERE` умові з цієї таблиці відбираються записи.
3. Потім вибрані дані групуються та агрегуються за допомогою `GROUP BY`.
4. Далі з агрегованих записів відбираються ті, які задовольняють умову `HAVING`.
5. Тільки після цього відповідно до зазначених у `SELECT` інструкціях формується результуюча таблиця — проводяться всі необхідні обчислення, надаються нові імена тощо.
6. Потім результуюча таблиця сортується відповідно до `ORDER BY`.
7. І нарешті спрацьовує обмеження кількості рядків, вказане в `LIMIT`.

Насправді це дуже важлива інформація, яку слід пам'ятати при складанні будь-яких SQL-запитів.

Можливо, ви вже зіткнулися з помилками, коли намагалися в блоці `WHERE` використовувати фільтрацію по нових полях, розрахованих у `SELECT`. Така операція не відповідає порядку виконання операторів і тому так робити не можна. Також ви могли потрапити в ситуацію, коли при фільтрації у `HAVING` у вас не приймався аліас колонки, розрахованої в `SELECT`. Це відбувалося з тієї ж причини – база даних просто ще не знала про те, що ви надали колонці нове ім'я.

І ще одна важлива порада. Зверніть увагу, що фільтрацію даних за неагрегованими значеннями можна робити як у блоці `WHERE`, так і в блоці `HAVING`. Уважно подивіться на такі запити:

```sql
SELECT sex, COUNT(user_id)
FROM users
WHERE sex != 'male'
GROUP BY sex


SELECT sex, COUNT(user_id)
FROM users
GROUP BY sex
HAVING sex != 'male'
```

Їх результат буде однаковим (можете переконатися в цьому самі).

Проте робити фільтрацію за неагрегованими даними рекомендується саме у блоці `WHERE`, тобто заздалегідь. У такому разі ви ще до угруповання прибираєте з розрахунків непотрібні вам дані і таким чином не витрачаєте обчислювальні ресурси на підрахунок значень, які будуть відфільтровані вами пізніше.

Це важливий момент щодо оптимізації SQL-запитів, тому рекомендуємо вам взяти до уваги інформацію.

## Підзапити

Підзапити це потужним інструментом в мові SQL, який дозволяє нам виконувати запити в середині інших запитів. Вони дають можливість створювати складніші і більш гнучкі запити, які вимагають доступу до даних з кількох таблиць або виконання додаткових обчислень.

Один із сценаріїв використання підзапитів - це пошук даних в одній таблиці на основі значень з іншої таблиці. Наприклад, ми можемо створити запит, що повертає всіх клієнтів, які зробили замовлення на товари з певної категорії, використовуючи підзапит для вибірки ідентифікаторів цих товарів з таблиці товарів.

Наприклад у нас є дві таблиці: `clients` і `orders`. Нам потрібно отримати список клієнтів, які зробили замовлення після певної дати. Для цього ми можемо скористатися підзапитом:

```sql
SELECT *
FROM clients
WHERE ID IN (SELECT client_id
             FROM orders
             WHERE data > '2023-01-01');
)
```

Іншими словами, підзапит — це лише запит усередині іншого запиту.

Підзапити можуть застосовуватись у таких частинах основного запиту:

- в операторі `FROM`;
- в операторі `SELECT` (якщо запит повертає один стовпець з одним значенням);
- в операторах `WHERE` та `HAVING` (якщо запит повертає один стовпець з одним або декількома значеннями).

Але давайте про все по порядку.

Насамперед важливо зрозуміти, що з результату виконання підзапиту можна звертатися також як і до таблиць у базі даних, тобто використовувати їх у блоці `FROM` замість наявних таблиць:

```sql
SELECT column_1
FROM (
    SELECT column_1, column_2
    FROM table
) AS subquery_1
```

В межах даного запиту спочатку буде виконано підзапит, який відбере колонки `column_1` і `column_2` з таблиці `table`, а потім вже з таблиці, що утворилася, основний запит вибере колонку `column_1`.

Важливий момент: при використанні підзапиту в блоці `FROM` сформованій на основі підзапиту таблиці необхідно привласнити якийсь аліас, інакше основний запит не спрацює. У прикладі вище ми позначили результат підзапиту `subquery_1`.

Крім того, рівнів вкладеності може бути декілька:

```sql
SELECT column_1
FROM (
    SELECT column_1, column_2
    FROM (
        SELECT column_1, column_2, column_3
        FROM table
    ) AS subquery_1
) AS subquery_2
```

В даному випадку послідовність роботи запиту така: спочатку буде виконано підзапит, що повертає результат `subquery_1`, потім підзапит, що повертає результат `subquery_2`, і тільки потім в результаті основного підзапиту потрапить колонка `column_1`. В результаті виходить щось схоже на матрьошку, при цьому до основної таблиці `table` звертається тільки перший підзапит `subquery_1`.

Наведений приклад досить умовний і на практиці колонки в такий спосіб відбирати не варто, але загальну ідею він має передавати.

Розуміння того, як працюють підзапити в блоці `FROM`, нам особливо стане в нагоді, коли ми вчитимемося об'єднувати різні таблиці.

А зараз давайте вирішимо просте завдання.

:::: {.callout-note icon=false}
## Завдання
::: {#exr-sql-subquery-01}
<br>
Використовуючи дані з таблиці `user_actions`, розрахуйте **середню кількість замовлень для всіх користувачів** нашого сервісу. Для цього спочатку в підзапиті порахуйте скільки замовлень зробив кожен користувач, а потім зверніться до результату підзапиту в блоці `FROM` і вже в основному запиті усередніть кількість замовлень по всіх користувачах. Отриману середню кількість замовлень всіх користувачів **округліть до двох знаків після коми**. Назвіть колонку з цим значенням `orders_avg`.

Поле у результуючій таблиці: `orders_avg`

**Пояснення:** до колонок з підзапиту можна застосовувати агрегатні функції — так само, як би ми зверталися до колонок вихідних таблиць.

:::

::::

In [2]:
#| code-fold: true
#| code-summary: "Рішення"

%%sql
SELECT round(avg(orders_count), 2) as orders_avg
FROM   (SELECT count(order_id) as orders_count
        FROM   user_actions
        WHERE  action = 'create_order'
        GROUP BY user_id) as t1

,orders_avg
0,2.78


## Табличні вирази: `WITH`

Вирішуючи попереднє завдання, ви могли запитати себе: а що якщо один і той же підзапит буде використовуватися в декількох частинах основного запиту? Невже щоразу доведеться дублювати той самий підзапит? А якщо рівнів вкладеності буде кілька? Чи не вийде тоді надто складний та громіздкий запит, який буде складно читати?

Для таких випадків SQL передбачений оператор `WITH`, який дозволяє створювати так звані **табличні вирази** (CTE, анг. *common table expressions*) - тимчасові таблиці, які існують тільки для одного запиту. Їхнє основне призначення полягає в розбиття складних запитів на кілька частин.

Табличні вирази створюються так:

```sql
WITH subquery_1 AS (
    SELECT column_1, column_2
    FROM table
)
SELECT column_1
FROM subquery_1
```

Порівняйте запит вище з результатом запиту, який ми використовували раніше:

```sql
SELECT column_1
FROM (
    SELECT column_1, column_2
    FROM table
) AS subquery_1
```

Оператор `WITH` може містити кілька табличних виразів, причому до зазначених раніше виразів можна звертатися у подальших виразах:

```sql
WITH subquery_1 AS (
    SELECT column_1, column_2, column_3
    FROM table
    ),
     subquery_2 AS (
    SELECT column_1, column_2
    FROM subquery_1
    )

SELECT column_1
FROM subquery_2
```

Можете знову порівняти запит вище із запитом, який ми використовували раніше:

```sql
SELECT column_1
FROM (
    SELECT column_1, column_2
    FROM (
        SELECT column_1, column_2, column_3
        FROM table
    ) AS subquery_1
) AS subquery_2
```

Використовувати у своїх запитах оператор `WITH` чи ні вирішувати вам, але в деяких випадках він може спростити роботу з кодом запиту.

::: {.callout-note}
Докладніше про `WITH` та табличні вирази можна почитати [тут](https://duckdb.org/docs/sql/query_syntax/with.html).
:::

:::: {.callout-note icon=false}
## Завдання
::: {#exr-sql-subquery-02}
<br>
Повторіть запит із [завдання @exr-sql-subquery-01], але тепер замість підзапиту використовуйте оператор `WITH` та табличний вираз. Умови завдання ті самі.

Поле у результуючій таблиці: `orders_avg`

:::

::::

In [3]:
#| code-fold: true
#| code-summary: "Рішення"

%%sql
with t1 as (SELECT user_id,
                   count(order_id) as orders_count
            FROM   user_actions
            WHERE  action = 'create_order'
            GROUP BY user_id)
SELECT round(avg(orders_count), 2) as orders_avg
FROM   t1

,orders_avg
0,2.78
